In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
import random
from tqdm.notebook import tqdm
from pathlib import Path
from collections import Counter
import requests
import json
from googlefinance import getQuotes
from bs4 import BeautifulSoup
import requests, sys, lxml.html
# from finquant.portfolio import build_portfolio

# import yfinance as yf
from yahooquery import Ticker

from src import URLS, extract_html, extract_table, load_etf_snapshot, \
    load_etf_analyst, load_etf_pie, load_etf_holdings, get_yf, get_ticker, \
    get_recommendations, get_fund_holdings, get_fund_sectors, get_type, get_fund_categories, \
    load_stock_snapshot, load_stock_contained
from src.fidelity import _stock_company

In [2]:
names = set(['MSFT', 'SNE', 'AMD', 'TSLA', 'NIO', 'LI', 
         'ARKK', 'VONG', 'IBB', 'SPY', 'ESPO', 'PHO', 'ACES', 'SCHO', 'LQD', 'ICVT', 
        'ACES', 'AGG', 'ASHR', 'BIDU'])
names.update(['ACES', 'AGG', 'AMD', 'AMZN', 'ASHR', 'BABA', 'BIDU', 'CIBR', 'CNYA', 'DIS', 'EMXC', 'ESPO', 
              'EWC', 'EWG', 'EWY', 'FB', 'FHLC', 'FIDU', 'FIPDX', 'FLJP', 'FNCL',
               'GOOG', 'IAGG', 'IAU', 'IBB', 'ICLN', 'IGOV', 'KWEB', 'LQD', 'MCHI', 
              'MINT', 'MSFT', 'NFLX', 'NIO', 'NTDOY', 'NVDA', 'PHO', 'PYPL', 'SCHO', 'SKYY', 'SNE', 'SOCL', 
              'TCEHY', 'TSLA', 'VCLT', 'VCSH', 'VGK', 'VGLT', 'VONG', 'XITK'])
names.update(['FSGGX', 'FSMAX', 'FSPSX', 'FUAMX', 'FZIPX', 'FZROX',])

names = list(names)

cats = {}
for n in names:
    t = get_type(n)
    if t not in cats:
        cats[t] = []
    cats[t].append(n)

name = random.choice(names)
name

'MINT'

In [3]:
cats

{'ETF': ['FIDU',
  'ASHR',
  'EWC',
  'KWEB',
  'SPY',
  'VCLT',
  'IBB',
  'VGLT',
  'VONG',
  'FHLC',
  'CNYA',
  'LQD',
  'SKYY',
  'EMXC',
  'XITK',
  'ICLN',
  'IAGG',
  'MCHI',
  'FNCL',
  'SOCL',
  'VCSH',
  'EWG',
  'CIBR',
  'IGOV',
  'SCHO',
  'VGK',
  'ESPO',
  'PHO',
  'MINT',
  'IAU',
  'AGG',
  'ACES',
  'EWY',
  'ARKK',
  'ICVT',
  'FLJP'],
 'EQUITY': ['AMZN',
  'NVDA',
  'SNE',
  'NFLX',
  'BIDU',
  'DIS',
  'NTDOY',
  'GOOG',
  'AMD',
  'LI',
  'TSLA',
  'PYPL',
  'MSFT',
  'NIO',
  'TCEHY',
  'FB',
  'BABA'],
 'MUTUALFUND': ['FSMAX', 'FZROX', 'FIPDX', 'FSGGX', 'FSPSX', 'FZIPX', 'FUAMX']}

In [102]:
# print(URLS['stocks']['snapshot'].format(ticker='GOOG'))
print(URLS['etfs']['composition'].format(ticker='SCHO'))
# for name in cats['EQUITY']:
# #     print(URLS['etf']['holdings'].format(ticker=etf))
#     print(URLS['stocks']['snapshot'].format(ticker=name))

https://eresearch.fidelity.com/eresearch/evaluate/snapshot.jhtml?symbols=GOOG
https://screener.fidelity.com/ftgw/etf/goto/snapshot/portfolioComposition.jhtml?symbols=SCHO


In [5]:
tk = get_ticker(name)
tk.symbols[0]

'EWG'

In [6]:
recs = get_recommendations(name)
recs

['EWU', 'EWQ', 'EWP', 'EWI', 'EWL']

In [35]:
get_type(name)

'ETF'

In [37]:
# [get_type(n) for n in names]

In [49]:
assets = get_fund_categories(name)
assets

{'stocks': 0.9958, 'bonds': 0.0}

In [3]:
# root = Path('raw/27-12-20/')
root = Path('raw/')
root = Path('raw/stocks')
root = Path('raw/dates/6-1-21/equity')
root

WindowsPath('raw/dates/6-1-21/equity')

In [7]:
# paths = list(root.glob('etf*'))
paths = sorted(list(root.glob('snapshot*')))
# paths = sorted(list(root.glob('*Details*')))
# paths = sorted(list(root.glob('etfholdings*')))
path = paths[18]
len(paths), path

(20, WindowsPath('raw/dates/6-1-21/equity/snapshot (9).html'))

In [8]:
bs = extract_html(path)

In [9]:
ticker = bs.findAll('title')[0].contents[0].split(' | ')[0].strip()
ticker

'DIS'

In [10]:
# bs

In [11]:
tbs = bs.findAll("table")
len(tbs)

33

In [20]:
table = tbs[13]

In [29]:

sector, industry = [x.text.strip() for x in table.find_all('span', {'class': 'right'})[:2]]

elms = table.find_all('p')
info = [el.text for el in elms]
start_idx = [i for i, x in enumerate(info) if x.startswith('Executives')][0] + 1
end_idx = [i for i, x in enumerate(info) if x.startswith('Environmental, Social, and')][0]
*people, addr, phone = elms[start_idx:end_idx]

In [30]:
sector, industry

('Communication Services', 'Entertainment')

In [31]:
start_idx, end_idx

(12, 14)

In [35]:
elms[8:]

[<p>The Walt Disney Company, together with its subsidiaries, operates as an entertainment company worldwide. The company's Media Networks segment operates domestic cable networks under the Disney, ESPN, Freeform, FX, and National Geographic brands; and television broadcast network under the ABC brand, as well as eight domestic television stations. This segment is also involved in the television production and distribution. Its Parks, Experiences and Products segment operates theme parks and resorts, such as Walt Disney World Resort in Florida; Disneyland Resort in California; Disneyland Paris; Hong Kong Disneyland Resort; and Shanghai Disney Resort; Disney Cruise Line, Disney Vacation Club, National Geographic Expeditions, and Adventures by Disney and Aulani, a Disney resort and spa in Hawaii, as well as licenses its intellectual property to a third party for the operations of the Tokyo Disney Resort in Japan. The company's Studio Entertainment segment produces and distributes motion p

In [18]:
company = _stock_company(tbs[13])

[<p>The Walt Disney Company (<span fmr-attach-component="fmr.qnr.components.snapshot.Mini" fmr-param-symbol="DIS">DIS</span>) will discuss fiscal first quarter 2021 financial results via a live audio webcast beginning at 4:30 p.m. ET / 1:30 p.m. PT on Thursday, February 11, 2021. Results will be released after the close of regular trading on February 11, 2021. To listen to the webcast, please visit www.disney.com/investors. The webcast presentation will be archived.</p>, <p>In a virtual gathering of chief executives convened to discuss political turbulence stemming from the 2020 election, leaders of some of the largest U.S. companies said they were considering withholding donations to Republican lawmakers seeking to impede the presidential transition. During an hourlong meeting Tuesday, a group of about three dozen chief executives discussed divisions in the country, efforts to thwart the presidential transition and what voice they...</p>, <p>http://archive.fast-edgar.com/20210105/ARBZ

TypeError: 'NoneType' object is not callable

In [113]:
# tb = tbs[25]
table = tbs[1]

In [171]:
extract_table(tbs[2])

[['Top 10', '21.46%'], ['Other', '78.54%']]

In [112]:
extract_table(table)

[['Asset Classification', 'Fixed Income'],
 ['Investment Philosophy', 'Passively Managed']]

In [99]:
# table

In [100]:
from src.fidelity import _fmt_human, _fmt_percent, _fmt_online

In [146]:
table

<table border="0" cellpadding="0" cellspacing="0">
<tr>
<th>Asset Classification</th>
<td>Fixed Income</td>
</tr>
<tr>
<th>Investment Philosophy</th>
<td>Passively Managed</td>
</tr>
</table>

In [46]:
# data = extract_table(table)

In [45]:
# bs.find_all('img', {'alt':"", 'height':"15", 'src':"https://www.fidelity.com/common/always-cached/etf/10-07/images/magnify2.gif", 
#                     'width':"17"})
els = bs.find('div', {'class':"spacing-div_15X0"}) # div class="spacing-div_15X0"
# els = bs.find_all('h3')
els

<div class="spacing-div_15X0"></div>

In [145]:
els[2]

<div class="header-with-borders">
<h3><a class="simple-popup" href="http://research2.fidelity.com/fidelity/screeners/etf/etfholdings.asp?symbol=SCHO&amp;view=Region" title="View Regions by Weight"><img alt="" height="15" src="https://www.fidelity.com/common/always-cached/etf/10-07/images/magnify2.gif" width="17"/> 
						Regional Exposure</a></h3>
</div>

In [168]:
missing = [x.text.strip().replace('Sorry : ', '').replace(' is not available for this symbol.', '') 
           for x in bs.find_all('div', {'class':"error-message"})]
missing

['Regional Exposure']

In [134]:
els[0].text.strip().split('\r')[0]

'Holdings'

In [139]:
tbs[4]

<table border="0" cellpadding="0" cellspacing="0" class="composition-table">
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#F7A600;"></div>
</td>
<th class="pie-sector-name" scope="row">Government</th>
<td class="pie-sector-area">99.86%</td>
</tr>
<tr class="alt-row-color">
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#003063;"></div>
</td>
<th class="pie-sector-name" scope="row">Corporate</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#E76108;"></div>
</td>
<th class="pie-sector-name" scope="row">Mortgage</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr class="alt-row-color">
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#529E00;"></div>
</td>
<th class="pie-sector-name" scope="row">Municipal</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr>
<td class="pie-se

In [141]:
tbs[5]

<table border="0" cellpadding="0" cellspacing="0" class="composition-table">
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#003063;"></div>
</td>
<th class="pie-sector-name" scope="row">United States</th>
<td class="pie-sector-area">100.00%</td>
</tr>
</table>

In [140]:
extract_table(tbs[4])

[['Government', '99.86%'],
 ['Corporate', '0.00%'],
 ['Mortgage', '0.00%'],
 ['Municipal', '0.00%'],
 ['Preferred', '0.00%']]

In [68]:
# for path in paths:
#     bs = extract_html(path)
#     tbs = bs.findAll("table")
#     tb = tbs[25]
#     rows = extract_table(tb)
#     print(rows[0])

In [40]:
txt = str(table)
offset = 0 #+ 91817
idx = txt[offset:].find('Robert A. Chapek')+offset
idx

13228

In [41]:
rn = 500
txt[idx-min(rn, idx):idx+min(rn,len(txt)-idx)]

'/eresearch/gotoBL/evaluate/snapshot.jhtml?symbols=DIS&amp;opt=1">Visit the new Stock Dashboard</a>\n</div>\n<div class="spacing-div_12X0"></div>\n<div class="dark-grey-hr"><div class="hr-for-ie"></div></div>\n<div class="spacing-div_15X0"></div>\n<div class="spacing-div_20X0"></div>\n<p><span class="bold">Full Time Employees:\xa0203,000</span></p>\n<div class="spacing-div_10X0"></div>\n<p><span class="bold">Executives</span></p>\n<div class="spacing-div_10X0"></div>\n<div>\n<ul>\n<div id="executives">\n<li>Mr. Robert A. Chapek,\xa0Chief Executive</li><li>Mr. Robert A. Chapek,\xa0Director</li><li>Mr. Robert A. Iger,\xa0Chairman</li><li>Mr. Robert A. Iger,\xa0Director</li><li>Ms. Christine Mary McCarthy,\xa0Chief Finance Officer</li>\n<div class="spacing-div_10X0"></div>\n<a href="javascript:viewMore(\'executives\');">\n<img height="12" src="https://www.fidelity.com/common/application/etf/14.10/images/plus_blue.gif" width="12"/>\r\n\t\t\t\t\t\tView more\r\n\t\t\t\t\t</a>\n</div>\n<div c

In [121]:
# len(str(tb))

In [120]:
# str(tb)[34000:35500]

In [173]:
root = Path('raw/stocks/')
# paths = root.glob('analyst*')
# paths = list(root.glob('snapshot*'))
paths = list(root.glob('snap*'))
len(paths)

17

In [152]:
# out = dict( load_etf_holdings(path) for path in tqdm(list(paths)))
out = dict( load_stock_snapshot(path) for path in tqdm(list(paths)))
len(out)

  0%|          | 0/17 [00:00<?, ?it/s]

17

In [31]:
# out['GOOG']

In [32]:
corps = {ticker:info['company'] for ticker, info in out.items()}
# corps

In [30]:
[c['name'] for c in corps.values()]

['Alphabet Inc',
 'Facebook Inc',
 'Advanced Micro Devices Inc',
 'Netflix Inc',
 'Nvidia Corp',
 'Amazoncom Inc',
 'Nintendo Co Ltd',
 'Li Auto Inc',
 'Microsoft Corp',
 'Walt Disney Co',
 'Alibaba Group Holding Ltd',
 'Tencent Holdings Ltd',
 'Nio Inc',
 'Tesla Inc',
 'Baidu Inc',
 'Paypal Holdings Inc',
 'Sony Corp']

In [188]:
root = Path('raw/27-12-20/')
# paths = root.glob('analyst*')
# paths = list(root.glob('snapshot*'))
# paths = list(root.glob('snap*'))
paths = list(root.glob('port*'))
path = paths[5]
len(paths), path

(36, WindowsPath('raw/27-12-20/portfolioComposition (14).html'))

In [189]:
# out = dict( load_etf_snapshot(path) for path in tqdm(list(paths)))
out = dict( load_etf_pie(path) for path in tqdm(list(paths)))
len(out)

  0%|          | 0/36 [00:00<?, ?it/s]

36

In [190]:
out.keys()

dict_keys(['FIDU', 'IGOV', 'SOCL', 'FLJP', 'EWG', 'SCHO', 'VONG', 'VCLT', 'EMXC', 'CNYA', 'AGG', 'KWEB', 'MCHI', 'VGLT', 'CIBR', 'ASHR', 'ICVT', 'ICLN', 'SKYY', 'FNCL', 'EWC', 'IBB', 'SPY', 'ACES', 'IAU', 'ARKK', 'IAGG', 'MINT', 'ESPO', 'XITK', 'FHLC', 'EWY', 'VCSH', 'LQD', 'PHO', 'VGK'])

In [194]:
# ticker, data = load_etf_pie(path)
# ticker, data.keys()

In [196]:
out['SPY']

{'Asset Classification': 'Equity',
 'Investment Philosophy': 'Passively Managed',
 'Holdings': {'Apple Inc': 6.37,
  'Microsoft Corp': 5.39,
  'Amazon.com Inc': 4.49,
  'Facebook Inc': 2.21,
  'Alphabet Inc': 1.72,
  'Berkshire Hathaway Inc': 1.51,
  'Johnson & Johnson': 1.27,
  'JPMorgan Chase & Co': 1.2,
  'Visa Inc': 1.18},
 'Sector Exposure': {'Information Technology': 27.54,
  'Health Care': 13.61,
  'Consumer Discretionary': 11.35,
  'Communication Services': 11.0,
  'Financials': 10.39,
  'Industrials': 8.7,
  'Consumer Staples': 6.74,
  'Utilities': 2.88,
  'Materials': 2.69,
  'Real Estate': 2.54,
  'Energy': 2.29},
 'Industry Exposure': {'Software': 8.71,
  'Technology Hardware, Storage & Peripherals': 6.65,
  'Interactive Media & Services': 5.8,
  'IT Services': 5.54,
  'Semiconductors & Semiconductor Equipment': 5.23,
  'Internet & Direct Marketing Retail': 5.0,
  'Pharmaceuticals': 3.9,
  'Health Care Equipment & Supplies': 3.75,
  'Banks': 3.75,
  'Capital Markets': 2.73,